<a href="https://colab.research.google.com/github/cripedel/DeepLearning_projects/blob/DeepLearning_projects/Facemask_projects.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# First imports

## Initial global imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip '/content/drive/My Drive/Dataset.zip'

Archive:  /content/drive/My Drive/Dataset.zip
replace MaskDataset/test/10001.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!pip install --upgrade tensorflow

Requirement already up-to-date: tensorflow in /usr/local/lib/python3.6/dist-packages (2.3.1)


In [ ]:
import os
from datetime import datetime

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

SEED = 1234
tf.random.set_seed(SEED)

cwd = os.getcwd()

## Necessary to work with JSON and preprocessing

In [ ]:
import json
import shutil
import random

# Preprocessing the images

## Initial directories to work with

In [ ]:
# Defining the datasets directory
dataset_dir = os.path.join(cwd, 'MaskDataset')
training_dir = os.path.join(dataset_dir, 'training')
validation_dir = os.path.join(dataset_dir, 'validation')
test_dir = os.path.join(dataset_dir, 'test')

# Create validation directory if it doesn't exist
if not os.path.exists(validation_dir):
    os.makedirs(validation_dir)


    

## Defining images classes

In [ ]:
# Loading the classes of each image into the memory
train_classes_json_file_name = 'train_gt.json'
train_classes_json_directory = os.path.join(dataset_dir, train_classes_json_file_name)

data = {}

with open(train_classes_json_directory) as json_file:
    data = json.load(json_file)

## Preparing folders for Keras usage in a later moment

In [ ]:
# Creating folder for each class of image for training and validation datasets
classes = set(data.values())

for class_label in classes:
    class_training_dir = os.path.join(training_dir, str(class_label))
    class_validation_dir = os.path.join(validation_dir, str(class_label))
    if not os.path.exists(class_training_dir):
        os.makedirs(class_training_dir)
    if not os.path.exists(class_validation_dir):
        os.makedirs(class_validation_dir)

In [ ]:
# Assigning images to each training folder/class, avoiding to have the same image two times in the same folder
for entry in os.scandir(training_dir):
    if(entry.is_file()):
        file_destination = os.path.join(training_dir, str(data[entry.name]), entry.name)
        if not os.path.isfile(file_destination):
            shutil.copy(entry.path, file_destination)
    
# Choosing random images to be into the validation folders, being able to repeat without cloning images
validation_rate = 0.1

for class_label in classes:
    class_training_dir = os.path.join(training_dir, str(class_label))
    class_validation_dir = os.path.join(validation_dir, str(class_label))
    
    for old_entry in os.scandir(class_validation_dir):
        os.remove(old_entry.path)
    
    training_entries = list(os.scandir(class_training_dir))
    validation_size = round(len(training_entries)*validation_rate)
    
    for validation_entry in random.sample(training_entries, validation_size):
        destination = os.path.join(class_validation_dir, validation_entry.name)
        os.rename(validation_entry.path, destination)

# Data augmentation

## Data generation

In [ ]:
x = tf.keras.preprocessing.image_dataset_from_directory(
    training_dir,
    labels="inferred",
    label_mode="int",
    class_names=None,
    color_mode="rgb",
    batch_size=8,
    image_size=(256, 256),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
)


Found 5052 files belonging to 3 classes.


In [ ]:
ls MaskDataset/training/



SyntaxError: ignored

In [ ]:
from keras.applications import imagenet_utils

tf.keras.applications.imagenet_utils.preprocess_input(x)

TypeError: ignored

In [ ]:
apply_data_augmentation = True

if apply_data_augmentation:
    train_data_gen = ImageDataGenerator(
        rotation_range=10,
        width_shift_range=10,
        height_shift_range=10,
        zoom_range=0.3,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='constant',
        cval=0,
        rescale=1/255.
    )
else:
    train_data_gen = ImageDataGenerator(rescale=1/255.)

valid_data_gen = ImageDataGenerator(rescale=1/255.)
# test_data_gen = ImageDataGenerator(rescale=1/255.)

## Data loader from directory

In [ ]:
bs = 8

train_gen = train_data_gen.flow_from_directory(
    training_dir,
    batch_size=bs,
    class_mode='categorical',
    shuffle=True,
    seed=SEED
)


valid_gen = valid_data_gen.flow_from_directory(
    validation_dir,
    batch_size=bs,
    class_mode='categorical',
    shuffle=True,
    seed=SEED
)

# test_gen = test_data_gen.flow_from_directory(
#     test_dir,
#     batch_size=bs,
#     class_mode='categorical',
#     shuffle=True,
#     seed=SEED
# )

In [ ]:
tf.keras.applications.imagenet_utils.preprocess_input?

In [ ]:

from keras.applications import imagenet_utils
img_h = 256
img_w = 256

num_classes = len(classes)

train_dataset = tf.data.Dataset.from_generator(
    lambda: train_gen,
    output_types=(tf.float32, tf.float32),
    output_shapes=([None, 256, 256, 3], [None, num_classes])
)

type(train_dataset)


train_dataset = train_dataset.repeat()

tf.keras.applications.imagenet_utils.preprocess_input(
    train_gen, data_format=None, mode='caffe'
)

type(train_dataset)

valid_dataset = tf.data.Dataset.from_generator(
    lambda: valid_gen,
    output_types=(tf.float32, tf.float32),
    output_shapes=([None, 32, 32, 3], [None, num_classes])
)

valid_dataset = valid_dataset.repeat()

# test_dataset = tf.data.Dataset.from_generator(
#     lambda: test_gen,
#     output_types=(tf.float32, tf.float32),
#     output_shapes=([None, 256, 256, 3], [None, num_classes])
# )

# test_dataset = test_dataset.repeat()
type(train_dataset)

## Testing the dataset

In [ ]:
iterator = iter(train_dataset)

In [ ]:
augmented_img, target = next(iterator)

augmented_img = np.array(augmented_img[0]) # it was a tensor, so we need to transform it into an array
augmented_img = augmented_img * 255 # normalizing

plt.imshow(np.uint8(augmented_img))
plt.plot()

# CNN part ;)

## Building the network

In [ ]:
from keras.datasets import cifar10
from keras.applications import DenseNet121
from keras.applications import imagenet_utils
from tensorflow.keras import optimizers
from keras.layers import Dropout, Flatten, Dense
import matplotlib.pyplot as plt
from keras.engine import Model
from sklearn.metrics import classification_report
import numpy as np
from keras.layers import Dropout
input_shape = (256, 256, 3)
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=input_shape)
for layer in base_model.layers: 
  #layer.trainable = False
  #print('Capa ' + layer.name + ' congelada...')
  last = base_model.layers[-1].output
x = Flatten()(last)
x = Dense(1000, activation='relu', name='fc1')(x)
x = Dropout(0.3)(x)
x = Dense(200, activation='relu', name='fc2')(x)
x = Dense(3, activation='softmax', name='predictions')(x)
model = Model(base_model.input, x)

In [ ]:
model.summary()

## Compiling the model

In [ ]:
loss = tf.keras.losses.CategoricalCrossentropy()

lr = 1e-4

optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

metrics = ['accuracy']

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/drive/My\ Drive/Keras3/classification_experiments/

In [ ]:
import os   #El codigo es siempre igual ? 
from datetime import datetime
#Start f ; caracteristicas iniciales 
cwd = os.getcwd()

exps_dir = os.path.join('/content/drive/My Drive/Keras3/', 'classification_experiments')
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)

now = datetime.now().strftime('%b%d_%H-%M-%S')

model_name = 'CNN'

exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)
    
callbacks = []

# Model checkpoint
# ----------------
ckpt_dir = os.path.join(exp_dir, 'ckpts')
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp_{epoch:02d}.ckpt'), 
                                                   save_weights_only=True)  # False to save the model directly
callbacks.append(ckpt_callback)

# Visualize Learning on Tensorboard
# ---------------------------------
tb_dir = os.path.join(exp_dir, 'tb_logs')
if not os.path.exists(tb_dir):
    os.makedirs(tb_dir)
    
# By default shows losses and metrics for both training and validation
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                             profile_batch=0,
                                             histogram_freq=1)  # if 1 shows weights histograms
callbacks.append(tb_callback)

# Early Stopping
# --------------
early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
    callbacks.append(es_callback)

In [ ]:
model.fit(
    x=train_dataset,
    epochs=40,
    steps_per_epoch=len(train_gen),
    validation_data=valid_dataset,
    validation_steps=len(valid_gen),
    callbacks=callbacks
)

In [ ]:
exps_dir = os.path.join(cwd, 'classification_experiments')
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)

now = datetime.now().strftime('%b%d_%H-%M-%S')

model_name = 'CNN'

exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)

callbacks = []

# Model checkpoint
# ----------------
ckpt_dir = os.path.join(exp_dir, 'ckpts')
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

ckpt_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=os.path.join(ckpt_dir, 'cp_.ckpt'), #'cp_{epoch:02d}.ckpt'
    save_weights_only=True # False to save the model directly
)
callbacks.append(ckpt_callback)

# Visualize Learning on Tensorboard
# ---------------------------------
tb_dir = os.path.join(exp_dir, 'tb_logs')
if not os.path.exists(tb_dir):
    os.makedirs(tb_dir)

# By default shows losses and metrics for both training and validation
tb_callback = tf.keras.callbacks.TensorBoard(
    log_dir=tb_dir,
    profile_batch=0,
    histogram_freq=1 # if 1 shows weights histograms
)
callbacks.append(tb_callback)

# Early Stopping
# --------------
early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
    callbacks.append(es_callback)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir 'D:\Polimi\Lectures\3semester\artificial-neural-networks-and-deep-learning\Homeworks\1st\classification_experiments\'